<a href="https://colab.research.google.com/github/aidanwoodring/RandomBusinessNameGenerator/blob/main/Midterm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
amazon_titles = pd.read_csv("amazon_prime_titles1.csv")

In [ ]:
titles = pd.concat([amazon_titles], axis = 0).reset_index()
titles.drop(['index'], axis=1, inplace=True)

In [ ]:
titles[titles.duplicated() == True].head(5)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score


In [ ]:
titles.drop_duplicates(inplace=True)

In [ ]:
# For genres
titles['genres'] = titles['genres'].str.replace(r'[','').str.replace(r"'",'').str.replace(r']','')
titles['genre'] = titles['genres'].str.split(',').str[0]

# For countries
titles['production_countries'] = titles['production_countries'].str.replace(r"[", '').str.replace(r"'", '').str.replace(r"]", '')
titles['production_country'] = titles['production_countries'].str.split(',').str[0]


<ipython-input-236-32d98c218ddc>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  titles['genres'] = titles['genres'].str.replace(r'[','').str.replace(r"'",'').str.replace(r']','')
<ipython-input-236-32d98c218ddc>:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  titles['production_countries'] = titles['production_countries'].str.replace(r"[", '').str.replace(r"'", '').str.replace(r"]", '')


In [ ]:
titles.drop(['genres', 'production_countries'], axis=1, inplace=True)

In [ ]:
titles['genre'] = titles['genre'].replace('', np.nan)
titles['production_country'] = titles['production_country'].replace('',np.nan)

In [ ]:
titles['seasons'].fillna(0, inplace=True)

In [ ]:
titles.drop(['imdb_id','age_certification'], axis=1,inplace=True)

In [ ]:
titles.dropna(inplace=True)

In [ ]:
lt = []
for i in titles['id']:
    movie_streaming = []
    if i in amazon_titles['id'].values:
        movie_streaming.append('amazon')
    lt.append(movie_streaming)

In [ ]:
titles['streaming_platform'] = lt

In [ ]:
movies = titles[titles['type'] == 'MOVIE'].copy().reset_index()
movies.drop(['index'], axis=1, inplace=True)

shows = titles[titles['type'] == 'SHOW'].copy().reset_index()
shows.drop(['index'], axis=1, inplace=True)

In [ ]:
def get_title(title,indices):
    """
    Function that gets the 'index searcher' and searches
    the user's title index.
    """
    
    try:
        index = indices[title]
    except:
        print("\n  Title not found")
        return None

    if isinstance(index, np.int64):
        return index
    
    else:
        rt = 0
        print("Select a title: ")
        for i in range(len(index)):
            print(f"{i} - {movies['title'].iloc[index[i]]}", end=' ')
            print(f"({movies['release_year'].iloc[index[i]]})")
        rt = int(input())
        return index[rt]

In [ ]:
def get_recommendations_movie(title, cosine_sim=cosine_sim_movies):
    """
    A function that takes a movie title as input and prints on the screen
    the 10 most similar movies based on the input description.
    """    
    
    title = get_title(title, indices_movies)
    if title == None:
        return 
    
    idx = indices_movies[title]
      
    print(f"Title: {movies['title'].iloc[idx]} |  Year: {movies['release_year'].iloc[idx]}")

    print('**' * 40)

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    print(movies[['title', 'release_year','streaming_platform']].iloc[movie_indices])

    print('**' * 40)

In [ ]:
def get_recommendations_show(title, cosine_sim=cosine_sim_shows):
    """
    A function that takes a show title as input and prints on the screen
    the 10 most similar shows based on the input description.
    """
    
    title = get_title(title, indices_shows)
    if title == None:
        return 
    
    idx = indices_shows[title]

    print(f"Title: {shows['title'].iloc[idx]} | Year: {shows['release_year'].iloc[idx]}")

    print('**' * 40)

    # Get the pairwsie similarity scores of all shows with that show
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the shows based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar shows
    sim_scores = sim_scores[1:11]

    # Get the show indices
    show_indices = [i[0] for i in sim_scores]

    print(shows[['title', 'release_year', 'streaming_platform']].iloc[show_indices])

    print('**' * 40)

In [252]:
get_recommendations_movie('Rocky')

Title: Rocky |  Year: 1976
********************************************************************************
                        title  release_year streaming_platform
3700                    Creed          2015           [amazon]
1365                 Rocky IV          1985           [amazon]
2002             Rocky Balboa          2006           [amazon]
1372                  Rocky V          1990           [amazon]
757                  Rocky II          1979           [amazon]
1367                Rocky III          1982           [amazon]
372          Broadway Limited          1941           [amazon]
6361         K.G.F: Chapter 2          2022           [amazon]
1263      The Joe Louis Story          1953           [amazon]
5001  Hello Guru Prema Kosame          2018           [amazon]
********************************************************************************


In [253]:
get_recommendations_movie('The Hungover Games')

Title: The Hungover Games |  Year: 2014
********************************************************************************
                                 title  release_year streaming_platform
3516                The Starving Games          2013           [amazon]
5076     Improv All Stars: Games Night          2018           [amazon]
6128                       Under World          2019           [amazon]
1073                 The Blue Gardenia          1953           [amazon]
6647  Federica Pellegrini - Underwater          2022           [amazon]
6517                          The Hunt          2021           [amazon]
2360     Ast√©rix at the Olympic Games          2008           [amazon]
4252           Plato's Reality Machine          2015           [amazon]
6375                              Dive          2022           [amazon]
6978       Mind Games - The Experiment          2023           [amazon]
********************************************************************************


In [ ]:
get_recommendations_show('This is a Test')


  Title not found


In [ ]:
get_recommendations_movie('This is a Test')


  Title not found
